In [1]:
import signal

In [2]:
import time
import io
from PIL import Image
import hashlib
import requests
import os
import pandas as pd
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver

In [3]:
search_term = "football"
num_images = 50

In [4]:
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH = "/home/brad/Documents/scraping/chromedriver_linux64/chromedriver"
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [5]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [6]:
def persist_image(folder_path,url):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [7]:
def handler(signum, frame):
    print("Forever is over!")
    raise Exception("end of time")

In [8]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    wd = webdriver.Chrome(executable_path=driver_path)
    res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
    
    #signal.signal(signal.SIGALRM, handler)
    
    for i, elem in enumerate(res):
        #signal.alarm(20)
        try:
            persist_image(target_folder,elem)
        except Exception as exc:
            print(str(i) + " took too long")
            print(exc)
    
    wd.quit()

In [9]:
search_and_download(search_term=search_term,
                   driver_path=DRIVER_PATH,
                   number_images=num_images)

Found: 100 search results. Extracting links from 0:100
Found: 51 image links, done!
SUCCESS - saved https://hokiesports.com/images/2020/5/29/ball_16FB_lu_DK_1307E.jpg - as ./images/football/eaef36694f.jpg
SUCCESS - saved https://townsquare.media/site/380/files/2019/04/Football-generic.jpg?w=980&q=75 - as ./images/football/da6565ef9e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRScvHJR_eC-28eQzm1N93v-cNZIQ96lVDP4w&usqp=CAU - as ./images/football/43778d2bb3.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRqcyuXR13ktnEL8197rnwjSVeeqxXy09kJgA&usqp=CAU - as ./images/football/0432d287f5.jpg
SUCCESS - saved https://www.al.com/resizer/aMAV3u0xmkHfc0uNkOVbIEcZxXM=/450x0/smart/cloudfront-us-east-1.images.arcpublishing.com/advancelocal/IHEQ7CJUVRGWTJ4BDYXYSNRHVQ.jpg - as ./images/football/38ede51301.jpg
SUCCESS - saved https://sportsnaut.com/wp-content/uploads/2020/04/USATSI_13846797_168386999_lowres-scaled.jpg - as ./images/football/958b

SUCCESS - saved https://img.bleacherreport.net/cms/media/image/a9/e6/b8/8f/b4ba/4101/90f4/bff7c523eaf4/crop_exact_Sanch.jpg?q=90&w=1440&h=960 - as ./images/football/33a01fb35f.jpg
SUCCESS - saved https://static.clubs.nfl.com/image/private/t_editorial_landscape_12_desktop/ravens/zayzsqxqudlxtbsifpp9 - as ./images/football/1da5346200.jpg
SUCCESS - saved https://i.ytimg.com/vi/8lvpn2K1_-o/maxresdefault.jpg - as ./images/football/b107c44010.jpg
SUCCESS - saved https://sportshub.cbsistatic.com/i/r/2019/01/02/1682c482-3750-4230-a8e4-56bd644565d8/thumbnail/1200x675/30a8f52789b0a375af42f72619f91904/cfp-logo-football.jpg - as ./images/football/6d472fb12c.jpg
SUCCESS - saved https://www.gannett-cdn.com/presto/2019/12/29/USAT/c8d166b5-fef2-4c7f-9959-7c16472a9a35-USATSI_13846795.jpg?crop=3473,1954,x1,y215&width=3200&height=1801&format=pjpg&auto=webp - as ./images/football/6c8dd7ccde.jpg
